In [40]:
# Import dependencies
import torch 
from PIL import Image    # used to load image
from torch import nn, save, load 
from torch.optim import Adam
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor

In [41]:
# get datasets  
# save to data folder, download True, train True, data transform to tensor
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())# dataset = DataLoader(train, 32)   # batch size 32
dataset = DataLoader(train, 32)   # convert batch into 32 images
# 1,28,28 - classes 0-9

In [42]:
# Image Classifier Neural Network 
# subclass nn.Module, implement __init__ and __forward__ methods
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()     # subclass model
        # create model using Sequencetial API
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),   # input channel, 1=black/white, 32 filters or kernels, shape of 3 by 3 
            nn.ReLU(),     # activation
            nn.Conv2d(32, 64, (3,3)),    # output 64 channels
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),            
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10)  # 64 channels *(28 - 6(shaving off 2 pixel at each time * 3 layers)) , need 10 outputs (0..9)
            )
    
    def forward(self, x):
        return self.model(x)

# Instance of the neural network, loss, optimizer 
clf = ImageClassifier().to('cpu') #.to('cpu')    create an instance 
opt = Adam(clf.parameters(), lr=1e-3)    # create optimizer Adam , learning rate 
loss_fn = nn.CrossEntropyLoss()     # loss function
            
def train():
    for epoch in range(5):    # 10 epoches
        for batch in dataset:    # batch data 
            X, y = batch 
            X, y = X.to('cpu'), y.to('cpu')
            yhat = clf(X)    # make prediction 
            loss = loss_fn(yhat, y)    # calculate loss_fn

            # apply backprop
            opt.zero_grad()    # zero out gradients 
            loss.backward()    # calculate gradients
            opt.step()     # apply gradient descents 

        print(f"Epoch:{epoch} loss is {loss.item()}")

    with open('model_state.pt', 'wb') as f:   # save model 
        save(clf.state_dict(), f)
    
def inference(image_file: str) -> str: 
    with open('model_state.pt', 'rb') as f:
        clf.load_state_dict(load(f))    # load weights into clf 
    img = Image.open(image_file)    # open image
    img_tensor = ToTensor()(img).unsqueeze(0).to('cpu')    # convert to tensor , unsqueeze for single image
    print(torch.argmax(clf(img_tensor)))

    
    
# Training flow, typical python function 
if __name__ == "__main__":
    # train model, comment this after training
#     train()
    
    # inference image for number 
    inference('./img_0.jpeg')
    inference('./img_2.jpeg')
    inference('./img_6.jpeg')
    inference('./img_9_to_1.jpeg')
    inference('./img_9.jpeg')
    


tensor(0)
tensor(2)
tensor(2)
tensor(4)
tensor(9)
